In [ ]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt

In [ ]:
import webbrowser

def open_website(k):
    website_url = f'https://fink-portal.org/{k}'
    webbrowser.open(website_url)


# Retrieve data

We begin by fetching the reduced data using the Python script `data_transfer.py` from the file `df_merged.parquet`, then importing it into Pandas data frame.

In [ ]:
df = pd.read_parquet('../scripts/df_merged1.parquet', engine='pyarrow')

In [ ]:
df.head(2)

In [ ]:
df.shape

Here we extract all unique IDs from our data frame:

In [ ]:
unique_ids = df['objectId'].unique().tolist()
len(unique_ids)

#

To calculate the weight values \(w_i\), we use the formula: `w_i` =\begin{cases}
\frac{1}{{\sigma_i^2}}, & \text{if data is available for day } i \\
0, & \text{otherwise}
\end{cases}

In [ ]:
missing_data = (df['source'] == 0) | (df['dc_sigflux'] == 0)
df['dc_weight'] = np.where(missing_data, 0, 1 / (df['dc_sigflux'] ** 2))
df['nr_weight'] = np.where(missing_data, 0, 1 / (df['nr_sigflux'] ** 2))

In [ ]:
df[['source','dc_weight']].head(3)

# 

# 

# With distance - factor test 

We group the data by shared ID and create `NumPy` arrays for flux, weighted flux, and the source test(if it's a missing day(data)). We also determine the length of each time series.

In [ ]:
grouped = df.groupby('objectId')

F = grouped['dc_flux'].apply(lambda x: x.values)#.values
sig = grouped['dc_sigflux'].apply(lambda x: x.values)#.values
W = grouped['dc_weight'].apply(lambda x: x.values)#.values
source = grouped['source'].apply(lambda x: x.values)#.values
lengths = grouped['source'].apply(lambda x: len(x))#.values


# We are not working with this right now because it causes a problem (
# the arrangement of values in these arrays might not be aligned perfectly with our expectations,
# especially if the grouping operation resulted in different lengths of arrays for different groups)

We define the length of our query,window or chunk, along with the limit factor and the size of each window or chunk.

In [ ]:
m=0
factor = 2*m+1 + 3*np.sqrt(2*(2*m+1))
chunk_size = 2 * (m + 1)

The `'no_match_test'` function evaluates whether there are any matches in the provided array, which contains the source test values for a window. If the sum of the array is less than or equal to 1, indicating that all values are missing or only one value is present, the function returns -99. Otherwise, it returns -1 to initialize the window's status as 'no match'.

In [ ]:
def no_match_test(array):
    if array.sum() <= 1 : 
        return -99 ## all are missing, or only one is
    return -1 # initialise as no match  # can be modifieted ? 

"`objects`" list contains a subset of the objects we intend to work with.

"`L_max`" is defined to facilitate partial iteration, serving as a debugging aid by allowing a limit to be set on the number of iterations performed.

We initialize the NumPy arrays with `None` values.

In [ ]:
objects = unique_ids#[0:3000]
num_objects = len(objects)
# L_max = int(num_objects/20)
L_max = 119
print("L_max ", L_max)

R = np.empty(num_objects, dtype=object)
Matches = np.empty(num_objects, dtype=object)
R_l = np.empty((num_objects, L_max), dtype=object)
num_by_obj = np.zeros((L_max,num_objects), dtype=object)
alp = np.empty((num_objects, L_max), dtype=object)
d = np.empty((num_objects, L_max), dtype=object)

In [ ]:
indexes_objects = range(num_objects)
Q = [None] * (L_max)
#num_by_Q = [0] * (L_max) ####### we don't need this one anymore <==> sum_i
selected_Q_K_i = np.empty(L_max, dtype=object)

We initialize R using the '`no_match_test`' function.

In [ ]:
for k in indexes_objects:
    
    n = lengths[objects[k]]
    num_chunks = int(n // 2)-m 

    #print(n,num_chunks)

    chunks = np.array([source[objects[k]][i*2 : (i*2+chunk_size)] for i in range(num_chunks)])
    result = np.array(list(map(no_match_test, chunks)))
    R[k] = result.copy()
    
    #Matches[k] = np.zeros(num_chunks, dtype=int)
    
    for l in range(L_max):
        R_l[k][l] = result.copy()
    # a for loop isn't a good idea !!!! 
    


### Loop to compute the distance of the subsequence in the time series to its nearest neighbor.

In [ ]:
start_time = time.time()

l= 0
while (l < L_max):
    indexes_array = np.array([np.where(array == -1)[0] for array in R], dtype=object)
    
    has_non_empty_list = np.any([value.size > 0 for value in indexes_array])
    if not has_non_empty_list:
        print("break , l = ", l )
        break
    
#     if l == 0:
#         Q[0]= [0]  *chunk_size
#     else:  
    for k in range(len(R)): # we can remove the loop for here ! ???
              if indexes_array[k].size != 0:
                f = F[objects[k]]

                index_no_match = indexes_array[k][0]

                #print(index_no_match, R[k])
                k_Query_taked = k
                selected_Q_K_i[l] = [k,index_no_match]

                Q[l] = f[index_no_match*2 : index_no_match*2 +chunk_size]
                break

            
    for k in range(len(objects)):
        f = F[objects[k]]
        w = W[objects[k]]
        n = lengths[objects[k]]
        n_c = n - 2*m # (number of chunks x 2) ! it's (n/2 - m) but to optimize we mutiply by 2 directly !  
        #print(n, n_c,len(R[k]))



        s_1 = np.zeros(n_c, dtype=float)
        s_2 = np.zeros(n_c, dtype=float)
        
        for j in range(0,m+1): 
            h = np.tile(Q[l][j*2: j*2+2], (len(f[j*2:j*2+ n_c]) // 2, 1)).ravel() # array of h for r and g successive for the vectorisation

            s_1[:] += (f[j*2:j*2+ n_c]*h*w[j*2:j*2+ n_c])
            s_2[:] += (h**2 * w[j*2:j*2+ n_c])


        s_n = s_1[::2] + s_1[1::2]  
        s_d = s_2[::2] + s_2[1::2] 
        
        mask_no_0 = (s_d != 0)
        alp[k][l] = np.zeros_like(s_d, dtype=float)

        alp[k][l][mask_no_0] = s_n[mask_no_0] / s_d[mask_no_0] # # Perform division only where s_d(i) is not zero

        alpha = np.repeat(alp[k][l], 2) # duplicate alpha for each value (one for r and second for g)
     
    
        dd = np.zeros(n_c, dtype=float)
        
        for j in range(0,m+1):
            h = np.tile(Q[l][j*2: j*2+2], (len(f[j*2:j*2+ n_c]) // 2, 1)).ravel() # array of h for r and g successive for the vectorisation
            
            dd[:] += ((f[j*2:j*2+ n_c] - alpha[:] * h)**2) * w[j*2:j*2+ n_c] 
            #alpha[:n-j*2] ==> alpha[:]

        d[k][l] = dd[::2] + dd[1::2]
        
        factor_comparison =  d[k][l] <= factor
                
        R[k][indexes_array[k][factor_comparison[indexes_array[k]]]] = l # explanation follows below!
        
        #count = 0
        for i in range(len(factor_comparison)):
            if factor_comparison[i] and R_l[k][l][i] != -99:
                R_l[k][l][i] = l
                #count += 1 
                #num_by_Q[l] +=1
                num_by_obj[l][k]+=1

                
        """if count >0:
            for i in range(len(factor_comparison)):
                  if factor_comparison[i] and R_l[k][l][i] != -99:# and i != index_no_match: it isn't bad but It may 
                        Matches[k][i] +=count"""
    
        
                
        """for i in indexes_array[k]:
            #print(i)
            if d[k][l][i] <= factor : 
                R[k][i] = l"""
#     print(count)
#     if count >0:
#             for k in range(len([R])):
#                 factor_comparison =  d[k][l] <= factor

#                 for i in range(len(factor_comparison)):
#                     if factor_comparison[i] and R_l[k][l][i] != -99:# and i != index_no_match: it isn't bad but It may 
#                         Matches[k][i] +=count
                
#     if l>0:
    R_l[k_Query_taked][l][index_no_match] = -2
#     R[k_Query_taked][index_no_match] = -2

    
     
    #print("l = ",l)
 
    l += 1 
    
print("l = ",l)



end_time = time.time()

# Compute the elapsed time
elapsed_time = end_time - start_time

print("Elapsed time:", elapsed_time, "seconds")

Let's break down the expression `R[k][indexes_array[k][factor_comparison[indexes_array[k]]]] = l` step by step:

1. `indexes_array[k]`: This selects the array of indexes corresponding to the k-th element of `indexes_array`.
2. `factor_comparison[indexes_array[k]]`: This applies boolean indexing to `factor_comparison` using the indexes from `indexes_array[k]`. It selects only the elements of `factor_comparison` corresponding to the indexes in `indexes_array[k]`.
3. `indexes_array[k][factor_comparison[indexes_array[k]]]`: This gives the indices where the condition `factor_comparison` is true for the k-th element of `indexes_array`.
4. `R[k][indexes_array[k][factor_comparison[indexes_array[k]]]]`: This uses the indices obtained in the previous step to select elements from the k-th row of `R`.
5. `= l`: Finally, it assigns the value `l` to the selected elements of `R[k]`.


In [ ]:
# np.save('array1.npy', d)
# np.save('array2.npy', R)
# np.save('array3.npy', Q[:-40])
# np.save('array4.npy', R_l)

# Load arrays individually when needed
# d = np.load('array1.npy', allow_pickle=True)
# R = np.load('array2.npy', allow_pickle=True)
# Q = np.load('array3.npy', allow_pickle=True)


In [ ]:
# np.savez('nested_arrays.npz', array1=d, array2=R, array3=Q[:], array4= R_l, array5 = selected_Q_K_i, array6= num_by_obj)
# np.savez('nested_arrays.npz', array1=d, array2=R, array3=Q[:], array4=R_l, array5=selected_Q_K_i, array6=num_by_obj, additional_variable1=m, additional_variable2=additional_variable2)
# Load the arrays from the .npz file
#data = np.load('nested_arrays.npz', allow_pickle=True)

# # Retrieve the arrays from the loaded data
# d1 = data['array1']
# R1 = data['array2']
# Q1 = data['array3']

In [ ]:
# import h5py

# # Assuming you have arrays d, R, Q, R_l, selected_Q_K_i, num_by_obj and additional variables additional_variable1, additional_variable2
# # Convert arrays to a supported dtype
# # d = d.astype(float)  # Example: converting to float
# R = R.astype(float)
# # Q = Q.astype(float)   # Example: converting to float
# R_l = R_l.astype(float)
# selected_Q_K_i = selected_Q_K_i.astype(float)
# num_by_obj = num_by_obj.astype(int)  # Example: converting to integer

# # Create an HDF5 file
# with h5py.File('nested_arrays.h5', 'w') as hf:
#     hf.create_dataset('array1', data=d)
#     hf.create_dataset('array2', data=R)
#     hf.create_dataset('array3', data=Q[:])
#     hf.create_dataset('array4', data=R_l)
#     hf.create_dataset('array5', data=selected_Q_K_i)
#     hf.create_dataset('array6', data=num_by_obj)
    
#     # Add additional variables as attributes
#     hf.attrs['additional_variable1'] = m
#     hf.attrs['additional_variable2'] = chunk_size


# 

# 

### Calculate the Matrix of Matches (with windows represented in rows and queries as columns)

In [ ]:
R[0], R[0]!=-99, sum(R[0]!=-99)

In [ ]:
rangek = range(len(objects))

# Initialize list to hold transposed arrays for each k
transposed_arrays_list = []

for k in rangek:  
    # Determine max length of arrays in R_l[k]
    max_length = max(len(arr) for arr in R_l[k])
    # we can remove this

    # Initialize transposed array
    transposed_array = np.empty((max_length, len(R_l[k])), dtype=object)

    # Transpose and store the arrays
    for i in range(max_length):
        for j, arr in enumerate(R_l[k]):
            if arr[i] == -2 or arr[i] >= 0:
                transposed_array[i][j] = 1
            elif arr[i] == -99 :
                transposed_array[i][j] = None
            else:
                transposed_array[i][j] = 0
                
                
    # Append transposed array to the list
    transposed_arrays_list.append(transposed_array)

transposed_arrays_list

In [ ]:
concatenated_array = np.concatenate(transposed_arrays_list, axis=0)
concatenated_array

In [ ]:
for i in concatenated_array: 
    print(i)
    print()

In [ ]:
sum_j = np.sum(concatenated_array, axis=1)
#sum_j

In [ ]:
sum_i = np.sum(concatenated_array, axis=0)
#sum_i

In [ ]:
max(sum_i),min(sum_i), np.argmax(sum_i), np.argmin(sum_i), selected_Q_K_i[94], selected_Q_K_i[100]

In [ ]:
#flattened_array = np.concatenate([arr.flatten() for arr in sum_j])
maximum_value = np.max(sum_j)
maximum_value, np.argmax(sum_j),sum_j[155]

In [ ]:
# Find indices of non-zero values
non_zero_indices = np.argwhere(sum_j != 0).flatten()

# Retrieve non-zero values
non_zero_values = sum_j[non_zero_indices]

# Find the index of the minimum non-zero value
min_non_zero_index = non_zero_indices[np.argmin(non_zero_values)]

min_non_zero_value = sum_j[min_non_zero_index]

min_non_zero_value, min_non_zero_index

In [ ]:
s = 0
global_inx = 633
def get_k_i_of_row(global_inx):
    if global_inx < (int(lengths[objects[0]]/2)-m):
         return  0 , global_inx
    for k in rangek:
        global_inx -= (int(lengths[objects[k]]/2)-m)
        if global_inx < (int(lengths[objects[k+1]]/2)-m):
            return  k+1 , global_inx
#def get
get_k_i_of_row(31)

In [ ]:
# transform to a function 
F[objects[10662]], F[objects[10662]][7*2:7*2+chunk_size], objects[10662], W[objects[10662]][7*2:7*2+chunk_size]

In [ ]:
def if_a_query(k_TS, i_TS, selected_Q_K_i):
    for index_Q, (k, i) in enumerate(selected_Q_K_i):
        if k_TS == k and i_TS == i:
            return True, index_Q, f"This window is used as the {index_Q}th query! "
    return False, 0 , "This window is not used as a query ! "

In [ ]:
num_by_Q[l]

In [ ]:
### max by j (as a window)
k , i = get_k_i_of_row(155)
l, string = if_a_query(k, i, selected_Q_K_i)
print(objects[k],"idx = ",i,"l =",l,".",string), plot_distance_flux(k,l)

### min by j (as a window)
k , i = get_k_i_of_row(global_inx)
l, string = if_a_query(k, i, selected_Q_K_i)
print(objects[k],"idx = ",i,"l =",l,".",string), plot_distance_flux(k,l)

k= 7683  ### 6 max by Q (as a Query )
i = 6 
l, string = if_a_query(k, i, selected_Q_K_i)
print(objects[k],"idx = ",i,"l =",l,".",string), plot_distance_flux(k,l)

k=10662 #### 7 min by Q (as a Query )
i = 7
l, string = if_a_query(k, i, selected_Q_K_i)
print(objects[k],"idx = ",i,"l =",l,".",string), plot_distance_flux(k,l)




In [ ]:
concatenated_array

In [ ]:
concatenated_array[0][0]#[0:3]

In [ ]:
len(sum_j), len(concatenated_array)

In [ ]:
get_k_i_of_row(122),get_j_by_k(4)

In [ ]:
get_k_i_of_row(155), get_k_i_of_Q(100),  

In [ ]:
def get_m(i,j):
    return concatenated_array[j][i]

def get_i_by_sum(i):
        return sum_i[i]

def get_j_by_sum(j):
        return sum_j[j]

    
def get_k_i_of_Q(l):
    return selected_Q_K_i[l]

def get_j_by_k(k_selected):
    global_inx = 0 
    for k in rangek:
        if k == k_selected:
            return list(range(global_inx, global_inx + int(lengths[objects[k]] / 2) - m))
        global_inx += (int(lengths[objects[k]] / 2) - m)



In [ ]:
total_matched = sum(sum_i) + sum(sum_j)
total_matched

In [ ]:
I_i_j = concatenated_array[:].copy() #*(sum_i[:])
I_i_j

In [ ]:
for j, sj in enumerate(sum_j):
    for i, si in enumerate(sum_i):
        dn = (si*sj)
        if dn == 0:
            I_i_j[j][i] *= np.inf
        else:
            I_i_j[j][i] *= total_matched/ dn


In [ ]:
I_i_j[0],I_i_j[4], I_i_j[1]

In [ ]:
nonzero_indices = np.nonzero(np.outer(sum_j, sum_i))

# Update only non-zero elements
I_i_j[nonzero_indices] = I_i_j[nonzero_indices] / (sum_i[nonzero_indices[1]] * sum_j[nonzero_indices[0]])

# For zero elements in dn, replace them with np.inf
#I_i_j[I_i_j == 0] = np.inf


In [ ]:
zero_indices = np.nonzero(~np.outer(sum_j, sum_i))
I_i_j[zero_indices]

#

#

#

#

#

## Functions to compute the numbers of matches and plot the results 

#### these functions should be in another file

We don't need this one anymore ! 

In [ ]:
def number_matches_for_all_k(R_l):
    all_matches = [] 
    
    # Iterate through each R_l[k]
    for R_l_k in R_l:
        # Get the length of the first sub-array
        length = len(R_l_k[0])

        # Initialize the list to store the number of matches for each index
        matches = [0] * length

        # Iterate through each sub-array
        for sub_array in R_l_k:
            # Iterate through each element and count matches for each index
            for i, elem in enumerate(sub_array):
                if elem >= 0 or elem ==-2:
                    matches[i] += 1

        all_matches.append(matches)  # Store matches for current k

    return all_matches

# all_matches will be a list where each element is matches for each k
# all_matches = number_matches_for_all_k(R_l)


In [ ]:
def lowest_values(all_matches):
    lowest_values = []  # Initialize a list to store the lowest 10 values

    # Iterate through each list of matches for each k
    for k, matches_k in enumerate(all_matches):
        # Iterate through each match count and index i
        for i, match_count in enumerate(matches_k):
            if R[k][i] !=-99:
                lowest_values.append((match_count, k, i))  # Store match count, k, and i

    # Sort the lowest values based on match count
    lowest_values.sort(key=lambda x: x[0])

    return lowest_values

# lowest_ will be a list of tuples where each tuple contains (match_count, k, i)
lowest = lowest_values(all_matches)
# lowest

In [ ]:
lowest

In [ ]:
alp[k][l].mean()

In [ ]:
def plot_distance_flux(k,l,open_Fink= False):
    
    plt.figure(figsize=(14, 12))

    # Plot for F[k]
    plt.subplot(2, 1, 1)

    for i in range(int(len(F[objects[k]])/2)):
        if source[objects[k]][2*i] == 0:
            marker = 'x'
            plt.errorbar(i, F[objects[k]][2*i], 
                     #sig[objects[k]][2*i]*0.4,
                     c='C0', marker=marker)
        else:
            marker = 'o'
            plt.errorbar(i, F[objects[k]][2*i], 
                     sig[objects[k]][2*i],
                     c='C0', marker=marker)

        if source[objects[k]][2*i+1] == 0:
            marker = 'x'
            plt.errorbar(i, F[objects[k]][2*i+1],
                     #sig[objects[k]][2*i+1]*0.4,
                     c='C1', marker=marker)
            
        else:
            marker = 'o'
            plt.errorbar(i, F[objects[k]][2*i+1],
                     sig[objects[k]][2*i+1],
                     c='C1', marker=marker)
            
        a_query,l_1, string = if_a_query(k,i,selected_Q_K_i)
            
        if a_query and l_1 == l:
            print(string)
            
            # Define the window of indices
            window_start = i  # Index of the window start
            window_end = i + int(len(Q[l])/2 - 1)  # Index of the window end

            # Create an array of float indices
            indices = np.arange(window_start, window_end + 1)
            indices = np.concatenate(([indices[0] - 0.5], indices, [indices[-1] + 0.5]))

            # Plot a shaded region for the window
            plt.fill_between(indices, min(Q[l])/1.2, max(Q[l])*1.2, color='gray', alpha=0.2)
            
        
    plt.plot([], [], color='C1', marker='x', label='missing points !')
    plt.plot([], [], color='C0', marker='o', label='origin')
    plt.plot([], [], color='C0', marker='x', label='missing points !')
    plt.plot([], [], color='C1', marker='o', label='origin')
    
    
    ################################### what about the seccond window !? 
    moy_alp = alp[k][l].mean()
    moy_alp = np.median(F[k])/np.median(Q[l]) #Manu test

    plt.plot(range(int(len(F[objects[k]])/2)), F[objects[k]][::2], c='C0', linewidth = 1)
    plt.plot(range(int(len(F[objects[k]])/2)), F[objects[k]][1::2], c='C1', linewidth = 1)

    plt.plot(np.arange(len(Q[l]) // 2) - 1-m, Q[l][::2]*moy_alp, c='g', label='Q[l]',marker='.', linewidth=3, zorder=3)
    plt.plot(np.arange(len(Q[l]) // 2) - 1-m, Q[l][1::2]*moy_alp, c='r', label='Q[l]',marker='.', linewidth=3, zorder=3)

    # Define the window of indices
    window_start = -1-m  # Index of the window start
    window_end = int(len(Q[l])/2 - 1-1 -m)  # Index of the window end

    # Create an array of float indices
    indices = np.arange(window_start, window_end + 1)
    indices = np.concatenate(([indices[0] - 0.4], indices, [indices[-1] + 0.4]))

    # Plot a shaded region for the window
    plt.fill_between(indices, min(Q[l])/1.2*moy_alp, max(Q[l])*1.2*moy_alp, color='gray', alpha=0.2)

    # plt.xlabel('Index')
    plt.ylabel('Flux')
    # plt.title('Flux Plot')
    plt.legend()
    




    ###############                       Plot for d[k][0]                       ###############

    plt.subplot(2, 1, 2)
    plt.plot([-1, -1], [0, 0], color='none')  # Plot an empty line with zero length
    plt.plot(range(len(d[k][l])), d[k][l],color='C0', linestyle='-',linewidth=1)
#     matches = all_matches[k]#Matches[k]#
#     for i, match_count in enumerate(matches):
#         plt.text(i+0.2, d[k][l][i]+d[k][l][i]*5/100, str(match_count), fontsize=10, color='red', ha='left')
    matches_idx = get_j_by_k(k)
    matches2 = sum_j[matches_idx]

    # Plot dummy points with desired colors and markers
    plt.plot([], [], color='black', marker='o', label='Query chosed')
    plt.plot([], [], color='red', marker='x', label='missing cases !')
    plt.plot([], [], color='blue', marker='s', label='Matched here')
    plt.plot([], [], color='green', marker='*', label='Matches with a different `l` (Query)')
    plt.plot([], [], color='yellow', marker='^', label=f'Not matched with any of the {L_max} options we selected')
    
    
    for i, val in enumerate(R_l[k][l]):
            if val == -99:
                plt.scatter(i, d[k][l][i], color='red', marker='x', s=50)  # marker size 50
                plt.text(i+0.2, d[k][l][i]+d[k][l][i]*5/100, str(matches2[i]), fontsize=10, color='red', ha='left')

            elif val == l:
                plt.scatter(i, d[k][l][i], color='blue', marker='s', s=50)
                plt.text(i+0.2, d[k][l][i]+d[k][l][i]*5/100, str(matches2[i]), fontsize=10, color='blue', ha='left')

            elif val == -2:
                plt.scatter(i, d[k][l][i], color='black', marker='o', s=50) 
                plt.text(i+0.2, d[k][l][i]+d[k][l][i]*5/100, str(matches2[i]), fontsize=10, color='black', ha='left')
            elif matches2[i]==0:
                plt.scatter(i, d[k][l][i], color='yellow', marker='^', s=50) 
                plt.text(i+0.2, d[k][l][i]+d[k][l][i]*5/100, str(matches2[i]), fontsize=10, color='yellow', ha='left')

            else:
                plt.scatter(i, d[k][l][i], color='green', marker='*', s=75)
                plt.text(i+0.2, d[k][l][i]+d[k][l][i]*5/100, str(matches2[i]), fontsize=10, color='green', ha='left')




    plt.legend(fontsize=8) 
    #plt.legend(loc='upper right', fontsize=8)

    plt.ylabel('distance')



    ###############                       Plot for alpha[k][l]                       ###############

#     plt.subplot(3, 1, 3)
#     plt.plot(range(len(alp[k][l])), alp[k][l], marker='.', linestyle='-',color='black')
#     plt.xlabel('Index')
#     plt.ylabel('Value')
#     plt.title('alpha Plot')




    plt.subplots_adjust(top=0.93)  # Adjust the top margin for the super title
    plt.suptitle(f"{objects[k]}, k = {k}, l = {l} ", fontname='Arial', fontsize=16, fontweight='bold')
    #plt.tight_layout()
    plt.show()
    

    if open_Fink:
        open_website(objects[k])




In [ ]:
plot_distance_flux(13653 ,0,True)
plot_distance_flux(22696 ,94,True)


In [ ]:
np.median(alp[0][0])

In [ ]:
np.median(F[0])/np.median(Q[100])


# 



# 



# 


# plot by multiple l

In [ ]:
k=3
l = 100
Q[l], F[objects][k][:2], Q[l]* np.median(alp[k][l]),np.median(alp[k][l]), Q[l]* (alp[k][l]).mean()

In [ ]:
l_values = [100,0]#32,33,1,4,5,12,15,26]#range(119)  #[0,1,2],2,3,4,5,6,7]#
k_values=range(len(objects))
k=1
for l in l_values:
    for k in k_values[:7]:
        plot_distance_flux(k,l) # True == open website


## plot histogram of numbers of matches for each Q

In [ ]:
indexes = range(len(sum_i))

# Plotting the histogram
plt.hist(indexes, bins=len(sum_i), weights=sum_i, color='skyblue', edgecolor='black')

# Adding labels and title
plt.xlabel('Indexes')
plt.ylabel('Numbers')
plt.title('Histogram of numbers of matches for each Q ')

# Display the histogram
plt.show()

In [ ]:
max(sum_i),min(sum_i)

In [ ]:
np.argmax(sum_i), np.argmin(sum_i)

## plot histogram of numbers of matches for a specific Q for all objects

In [ ]:
num_by_obj

In [ ]:
l=94
indexes = range(len(num_by_obj[l]))

# Plotting the histogram
plt.hist(indexes, bins=len(num_by_obj[l]), weights=num_by_obj[l], color='skyblue', edgecolor='black')

# Adding labels and title
plt.xlabel('Indexes')
plt.ylabel('Numbers')
plt.title('Histogram of numbers of matches for a specific Q for all objects')

# Display the histoazgram
plt.show()

In [ ]:
count_minus_99 = np.array([np.sum(subarr == -99) for subarr in R])

result = np.where(count_minus_99 <= 10, True, False)
result,len(result)

In [ ]:
num_by_obj[0][result]

In [ ]:
indices = np.where(result)[0]

sub_arrays = num_by_obj[0][indices]
sub_arrays, indices

In [ ]:
k = indices[np.argmax(sub_arrays)]
k

In [ ]:
plot_distance_flux(4,l)

In [ ]:
len(num_by_obj[l][indices])

In [ ]:
l=94
indexes = range(len(num_by_obj[l][indices]))

# Plotting the histogram
plt.hist(indices, bins=len(num_by_obj[l][indices]), weights=num_by_obj[l][indices], color='skyblue', edgecolor='black')

# Adding labels and title
plt.xlabel('Indexes')
plt.ylabel('Numbers')
plt.title('Histogram of numbers of matches for a specific Q for all objects')

# Display the histoazgram
plt.show()

### Compute the highest and the lowest indices 

In [ ]:
lowest_indices_array = []
highest_indices_array = []
sorted_indices_list = []

# Assuming num_by_obj is your list of arrays
for l in range(len(num_by_obj)):
    # Convert the array to a numpy array
    array = np.array(num_by_obj[l])

    # Get indices of sorted array in ascending order
    sorted_indices = np.argsort(array)
    sorted_indices_list.append(sorted_indices)  # Append sorted indices list

    # Indices of 10 lowest values
    lowest_indices = sorted_indices[:10]

    # Indices of 10 highest values
    highest_indices = sorted_indices[-10:][::-1]  # Reverse to get highest indices

    # Append indices to arrays
    highest_indices_array.append(highest_indices)
    
    lowest_values = array[lowest_indices[0]]
    all_lowest_indices = np.where(array == lowest_values)[0]
    lowest_indices_array.append(all_lowest_indices)


In [ ]:
sorted_indices_list[0][-10:][::-1]

In [ ]:
l=94
highest_indices_array[l],lowest_indices_array[l], len(lowest_indices_array[l])

In [ ]:
for i,k in enumerate(objects):
    if k == 'ZTF18cckcza':#'ZTF19acggsdc':
        print(i)
        break

In [ ]:
df[df['objectId'] == 'ZTF18cckcza']

In [ ]:
plot_distance_flux(k,l,True)


In [ ]:
objects[23683]

In [ ]:
lowest_indices_array[l], num_by_obj[l]

In [ ]:
l=94
count = 1
for k in sorted_indices_list[l][len(lowest_indices_array[l]):len(lowest_indices_array[l])+10]:
    print(count)
    count += 1
    print(objects[k],k,"l =",l,".")
    print("Number of matches in this object of this Q",num_by_obj[l][k])
    plot_distance_flux(k,l,True)
    

In [ ]:
sorted_indices_list

### 

### 

### plot Qr/Qg 

In [ ]:
Q[94]

In [ ]:
Q_array = np.array(Q)
ratios = Q_array[:, 0] / Q_array[:, 1]

plt.plot(ratios)
plt.title('Ratio of First Value to Second Value')
plt.xlabel('Index')
plt.ylabel('Ratio')
plt.grid(True)
plt.show()

In [ ]:
Q_array = np.array(Q)
ratios = Q_array[:, 0] / Q_array[:, 1]

plt.scatter(ratios[:],sum_i[:]) ###################### -3 to exclude the 800 .. 
plt.title('num of matches for each Q in function of Ratio ')
plt.xlabel('Ratio Qr/Qg')
plt.ylabel('num(l)')
plt.grid(True)
plt.show()

In [ ]:
Q

In [ ]:
selected_Q_K_i[116], Q[116]

In [ ]:
plot_distance_flux(23056,116,True)
print(objects[k])
print(objects[23056])

In [ ]:
ratios[np.argmax(sum_i)]

In [ ]:
ratios

#

# Distinguishing the two cases!

In [ ]:
m=1 # here the minimin is 1 !! 
factor = 2*m+1 + 3*np.sqrt(2*(2*m+1))
chunk_size = 2 * (m + 1)

In [ ]:
objects = unique_ids[0:10]
num_objects = len(objects)
L_max = int(num_objects/2)

print("L_max ", L_max)

R_r = np.empty(num_objects, dtype=object)
R_g = np.empty(num_objects, dtype=object)

R_l_r = np.empty((num_objects, L_max), dtype=object)
R_l_g = np.empty((num_objects, L_max), dtype=object)

alp = np.empty((num_objects, L_max), dtype=object)
d_r = np.empty((num_objects, L_max), dtype=object)
d_g = np.empty((num_objects, L_max), dtype=object)

In [ ]:
indexes_objects = range(num_objects)
Q = [None] * (L_max)

In [ ]:
for k in indexes_objects:

    n = lengths[objects[k]]
    num_chunks = int(n // 2)-m 
    
    chunks_g = np.array([source[objects[k]][i*2 : (i*2+chunk_size):2] for i in range(num_chunks)])
    chunks_r = np.array([source[objects[k]][1+i*2 : (i*2+chunk_size):2] for i in range(num_chunks)])
    result_r = np.array(list(map(no_match_test, chunks_r)))
    result_g = np.array(list(map(no_match_test, chunks_g)))
    
    R_r[k] = result_r.copy()
    R_g[k] = result_g.copy()
    
        
    for l in range(L_max):
        R_l_r[k][l] = result_r.copy()
        R_l_g[k][l] = result_g.copy()
    # a for loop isn't a good idea !!!! 
        

In [ ]:
start_time = time.time()


l= 0
while (l < L_max):
    indexes_array_r = np.array([np.where(array == -1)[0] for array in R_r], dtype=object)
    indexes_array_g = np.array([np.where(array == -1)[0] for array in R_g], dtype=object)
#     break
    empty_lists = np.all([value.size == 0 for value in indexes_array_r]) or np.all([value.size == 0 for value in indexes_array_g])
    if empty_lists:
        print("break , l = ", l )
        break
        
    for k in range(len(R_g)): # we can remove the loop for here ! ???
          if (indexes_array_g[k].size != 0):
            f = F[objects[k]]
            
            k_Query_taked_g = k 

            index_no_match_g = indexes_array_g[k][0] # here x 2 because we have the indexes only for g (I need to check this point)        
            Q[l] = f[index_no_match_g* 2  : index_no_match_g* 2  +chunk_size:2]
            break
            
    for k in range(len(R_r)): # we can remove the loop for here ! ???
          if (indexes_array_r[k].size != 0):
            f = F[objects[k]]
            k_Query_taked_r = k 

            index_no_match_r = indexes_array_r[k][0]  #+1
            for i in range(1, chunk_size, 2):
                 Q[l] = np.insert(Q[l], i, f[index_no_match_r*2 + i : index_no_match_r*2 + i + 1])

            #Q[l] = f[index_no_match : index_no_match +chunk_size]
            break    
    
    
    
    for k in range(len(objects)):
        f = F[objects[k]]
        w = W[objects[k]]
        n = lengths[objects[k]]
        n_c = n - 2*m # (number of chunks x 2) ! it's (n/2 - m) but to optimize we mutiply by 2 directly !  
        #print(n, n_c,len(R[k]))



        s_1 = np.zeros(n_c, dtype=float)
        s_2 = np.zeros(n_c, dtype=float)
        
        for j in range(0,m+1): 
            h = np.tile(Q[l][j*2: j*2+2], (len(f[j*2:j*2+ n_c]) // 2, 1)).ravel() # array of h for r and g successive for the vectorisation

            s_1[:] += (f[j*2:j*2+ n_c]*h*w[j*2:j*2+ n_c])
            s_2[:] += (h**2 * w[j*2:j*2+ n_c])

        s_n = s_1#[::2] + s_1[1::2]  # this needs to optimizate with new variables ! 
        s_d = s_2#[::2] + s_2[1::2] 
        
        mask_no_0 = (s_d != 0)
        alp[k][l] = np.zeros_like(s_d, dtype=float)

        alp[k][l][mask_no_0] = s_n[mask_no_0] / s_d[mask_no_0] # # Perform division only where s_d(i) is not zero

        alpha = alp[k][l]#np.repeat(alp[k][l], 2) # duplicate alpha for each value (one for r and second for g)
     
    
        dd = np.zeros(n_c, dtype=float)
        
        for j in range(0,m+1):
            h = np.tile(Q[l][j*2: j*2+2], (len(f[j*2:j*2+ n_c]) // 2, 1)).ravel() # array of h for r and g successive for the vectorisation
            
            dd[:] += ((f[j*2:j*2+ n_c] - alpha[:] * h)**2) * w[j*2:j*2+ n_c] 
            #alpha[:n-j*2] ==> alpha[:]

        d_g[k][l] = dd[::2]
        d_r[k][l] = dd[1::2]
        
        """factor_comparison =  d_g[k][l] <= factor
        R_g[k][indexes_array_g[k][factor_comparison[indexes_array_g[k]]]] = l # explanation follows below!
                
        factor_comparison =  d_r[k][l] <= factor
        R_r[k][indexes_array_r[k][factor_comparison[indexes_array_r[k]]]] = l # explanation follows below!"""
        

        for i in indexes_array_g[k]:
            if d_g[k][l][i] <= factor : 
                R_g[k][i] = l
        for i in indexes_array_r[k]:
            if d_r[k][l][i] <= factor : 
                R_r[k][i] = l
                
        factor_comparison = d_r[k][l] <= factor      
        
        for i in range(len(factor_comparison)):
            if factor_comparison[i] and R_l_r[k][l][i] != -99: 
                R_l_r[k][l][i] = l
            
        factor_comparison = d_g[k][l] <= factor      

        for i in range(len(factor_comparison)):
            if factor_comparison[i] and R_l_g[k][l][i] != -99: 
                R_l_g[k][l][i] = l
                
        """for i in indexes_array[k]:
            #print(i)
            if d[k][l][i] <= factor : 
                R[k][i] = l"""
                
    R_l_r[k_Query_taked_r][l][index_no_match_r] = -2
    R_l_g[k_Query_taked_g][l][index_no_match_g] = -2

    
    l += 1 
    
print("l = ",l)



end_time = time.time()

# Compute the elapsed time
elapsed_time = end_time - start_time

print("Elapsed time:", elapsed_time, "seconds")

#

In [ ]:
all_matches_r = number_matches_for_all_k(R_l_r)
all_matches_g = number_matches_for_all_k(R_l_g)


In [ ]:
k = 1
l = 8
unique_ids[k]

In [ ]:
color = ['green','red']
color_Q = ['C0','C1']

def plot_distance_flux_r_g(k,l,j=0,d=d_g,R_l=R_l_g,matches=all_matches_g[k]): # default case : g (j=0 ==> green)
    
    # # Plotting
    plt.figure(figsize=(15, 10))

    # Plot for F[k]
    plt.subplot(2, 1, 1)

    for i in range(int(len(F[objects[k]])/2)):
        if source[objects[k]][2*i+j] == 0:
            marker = 'x'
            plt.errorbar(i, F[objects[k]][2*i+j], 
                     #sig[objects[k]][2*i+j]*0.4,
                     c=color[j], marker=marker)
        else:
            marker = 'o'
            plt.errorbar(i, F[objects[k]][2*i+j], 
                         sig[objects[k]][2*i+j],
                         c=color[j], marker=marker)


    plt.plot([], [], color=color[j], marker='o', label='origin')
    plt.plot([], [], color=color[j], marker='x', label='missing points !')


    plt.plot(range(int(len(F[objects[k]])/2)), F[objects[k]][j::2], c=color[j], linewidth = 1)


    plt.plot(range(int(len(Q[l])/2)), Q[l][j::2], c=color_Q[j], label='Q[l]',marker='.')

    # Define the window of indices
    window_start = -0  # Index of the window start
    window_end = int(len(Q[l])/2 - 1)  # Index of the window end

    # Create an array of float indices
    indices = np.arange(window_start, window_end + 1)
    indices = np.concatenate(([indices[0] - 0.5], indices, [indices[-1] + 0.5]))

    # Plot a shaded region for the window
    plt.fill_between(indices, min(Q[l][j::2])/1.2, max(Q[l][j::2])*1.2, color='gray', alpha=0.2)

    plt.ylabel('Flux')
    plt.legend()

    # Plot for d[k][0]

    plt.subplot(2, 1, 2)
    plt.plot(range(len(d[k][l])), d[k][l],color='C0', linestyle='-',linewidth=1)

        # Plot dummy points with desired colors and markers
    plt.plot([], [], color='black', marker='o', label='Query chosed')
    plt.plot([], [], color='red', marker='x', label='missing cases !')
    plt.plot([], [], color='blue', marker='s', label='Matched here')
    plt.plot([], [], color='green', marker='*', label='Matches with a different `l` (Query)')
    plt.plot([], [], color='yellow', marker='^', label=f'Not matched with any of the {L_max} options we selected')

    for i, val in enumerate(R_l[k][l]):
            if val == -99:
                plt.scatter(i, d[k][l][i], color='red', marker='x', s=50)  # marker size 50
                plt.text(i+0.2, d[k][l][i]+d[k][l][i]*5/100, str(matches[i]), fontsize=10, color='red', ha='left')

            elif val == l:
                plt.scatter(i, d[k][l][i], color='blue', marker='s', s=50)
                plt.text(i+0.2, d[k][l][i]+d[k][l][i]*5/100, str(matches[i]), fontsize=10, color='blue', ha='left')

            elif val == -2:
                plt.scatter(i, d[k][l][i], color='black', marker='o', s=50) 
                plt.text(i+0.2, d[k][l][i]+d[k][l][i]*5/100, str(matches[i]), fontsize=10, color='black', ha='left')
            elif matches[i]==0:
                plt.scatter(i, d[k][l][i], color='yellow', marker='^', s=50) 
                plt.text(i+0.2, d[k][l][i]+d[k][l][i]*5/100, str(matches[i]), fontsize=10, color='yellow', ha='left')

            else:
                plt.scatter(i, d[k][l][i], color='green', marker='*', s=75)
                plt.text(i+0.2, d[k][l][i]+d[k][l][i]*5/100, str(matches[i]), fontsize=10, color='green', ha='left')




    plt.legend(fontsize=8) 

    plt.ylabel('distance')


    # # Plot for alphat[k][0]
    # plt.subplot(4, 1, 4)
    # plt.plot(range(len(alp[k][l][::2])), alp[k][l][::2], marker='.', linestyle='-',color='black')
    # plt.xlabel('Index')
    # plt.ylabel('alpha')
    # # plt.title('alpha Plot')


    plt.subplots_adjust(top=0.93)  # Adjust the top margin for the super title
    plt.suptitle(f"{objects[k]}, k ={k}, l ={l}", fontname='Arial', fontsize=16, fontweight='bold')

    plt.show()



In [ ]:
l_values = [0,1,2,3]#range(L_max)  
k= 0
for l in l_values:
#     plot_distance_flux_r_g(k,l,1,d_r,R_l_r,all_matches_r[k])
    plot_distance_flux_r_g(k,l,0,d_g,R_l_g,all_matches_g[k])
    

#

#

#

#

#

#

#

#

#

# TEST

In [ ]:
import numpy as np

# Given data
f = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8])
index_no_match = 4
chunk_size = 4

# Initialize Q[l]
Q = np.array([])

# First part
Q = f[index_no_match : index_no_match + chunk_size : 2]
index_no_match = 2
f = np.array([10, 11, 12, 13, 14, 15, 16, 17, 18,19])

# Second part
for i in range(1, chunk_size, 2):
    print(i)
    Q = np.insert(Q, i, f[index_no_match*2 + i : index_no_match*2 + i + 1])

# Print Q[l]
print(Q)
# 15 17 =2
# 13 15 =1

In [ ]:
list1 = [0,0,1,1,2,2,3,3,4,4,5,5,6,6,7,7,8,8,9,9]
list2 = [[0,0,1,1],[1,1,2,2],[2,2,3,3],[3,3,4,4],[4,4,5,5]]

In [ ]:
m=1
idx =3
chunk_size = 2 * (m + 1)
list1[idx*2:idx*2+chunk_size:2]

In [ ]:
m=1
idx =3
chunk_size = 2 * (m + 1)
list1[idx*2:idx*2+chunk_size]

In [ ]:
R = np.empty(len(objects), dtype=object)

m=1
chunk_size =2 * (m + 1)

for k in range(len(objects)):

    n = len(source[k])
    #num_chunks = n // chunk_size

    #chunks = [source[k][i*chunk_size : (i+1)*chunk_size] for i in range(num_chunks)]
    num_chunks = int(n // 2)-m 

    print(n,num_chunks)


    chunks = np.array([source[k][i*2 : (i*2+chunk_size)] for i in range(num_chunks)])
#     if m>0: 
#         for j in range(1, m+1):
#             chunks.extend([source[k][i*chunk_size +j*2 : (i+1)*chunk_size+j*2] for i in range(num_chunks-1)])
#     if m > 0:
#         for j in range(1, m + 1):
#             for i in range(num_chunks - 1):
#                 chunks.append(source[k][i * chunk_size + j * 2: (i + 1) * chunk_size + j * 2])

    #result = np.concatenate(np.array([no_match_test(chunk) for chunk in chunks]))  # Apply no_match_test directly
    result = np.array(list(map(no_match_test, chunks)))
    R[k] = result

    ### le

In [ ]:
m=0
for j in range(1, max(1, m+1)):
    print(j*2)

if m>0: 
    for j in range(1, m+1):
        print(j*2)

In [ ]:
import timeit

m = 100

code_snippet_1 = """
s_1 = np.zeros(n, dtype=float)
s_2 = np.zeros(n, dtype=float)
s_3 = np.zeros(n, dtype=float)
s_4 = np.zeros(n, dtype=float)
s_5 = np.zeros(n, dtype=float)
s_6 = np.zeros(n, dtype=float)
"""

code_snippet_2 = """
zeros = np.zeros(n, dtype=float)

s_1 = zeros.copy()
s_2 = zeros.copy()
s_3 = zeros.copy()
s_4 = zeros.copy()
s_5 = zeros.copy()
s_6 = zeros.copy()

"""

time_taken_1 = timeit.timeit(stmt=code_snippet_1, number=10000000, globals=globals())
time_taken_2 = timeit.timeit(stmt=code_snippet_2, number=10000000, globals=globals())

print("Time snippet 1:", time_taken_1)
print("Time snippet 2:", time_taken_2)



In [ ]:
import timeit
# First code snippet
def first_code(): ##### this is better ! 
    I_i_j = concatenated_array[:]
    for j, sj in enumerate(sum_j):
        for i, si in enumerate(sum_i):
            dn = (si * sj)
            if dn == 0:
                I_i_j[j][i] *= np.inf
            else:
                I_i_j[j][i] *= total_matched/ dn
# Second code snippet
def second_code():
    I_i_j = concatenated_array[:] * total_matched
    for j, sj in enumerate(sum_j):
        for i, si in enumerate(sum_i):
            dn = (si * sj)
            if dn == 0:
                I_i_j[j][i] *= np.inf
            else:
                I_i_j[j][i] /= dn

# Test and compare the performance
num_iterations = 1

time_first = timeit.timeit(first_code, number=num_iterations)
time_second = timeit.timeit(second_code, number=num_iterations)

print("Time taken by first code snippet:", time_first)
print("Time taken by second code snippet:", time_second)


#

#

#

# distance dict to dataframe 

In [ ]:
d

#

#

# Trash

In [ ]:
def plot_distance_flux_g(k,l):
    
    # # Plotting
    plt.figure(figsize=(15, 10))

    # Plot for F[k]
    plt.subplot(2, 1, 1)

    # plt.plot(range(int(len(F[k])/2)), F[k][::2], c='g', label='F[k] Even')
    #plt.plot(range(int(len(F[k])/2)), F[k][1::2], c='r', label='F[k] Odd')

    for i in range(int(len(F[objects[k]])/2)):
        if source[objects[k]][2*i] == 0:
            marker = 'x'
            plt.errorbar(i, F[objects[k]][2*i], 
                     #sig[objects[k]][2*i]*0.4,
                     c='g', marker=marker)
        else:
            marker = 'o'
            plt.errorbar(i, F[objects[k]][2*i], 
                         sig[objects[k]][2*i],
                         c='g', marker=marker)

    #     if source[k][2*i+1] == 0:
    #         marker = 'x'
    #     else:
    #         marker = 'o'
    #     plt.errorbar(i, F[k][2*i+1],
    #                  sig[k][2*i+1]*0.4,
    #                  c='r', marker=marker)
    # plt.plot([], [], color='red', marker='x', label='missing points !')
    plt.plot([], [], color='green', marker='o', label='origin')
    plt.plot([], [], color='green', marker='x', label='missing points !')
    # plt.plot([], [], color='red', marker='o', label='origin')


    plt.plot(range(int(len(F[objects[k]])/2)), F[objects[k]][::2], c='g', linewidth = 1)
    # plt.plot(range(int(len(F[k])/2)), F[k][1::2], c='r', linewidth = 1)


    plt.plot(range(int(len(Q[l])/2)), Q[l][::2], c='C0', label='Q[l]',marker='.')
    #plt.plot(range(int(len(Q[0])/2)), Q[l][1::2], c='C1', label='Q[l]',marker='.')

    # Define the window of indices
    window_start = -0  # Index of the window start
    window_end = int(len(Q[l])/2 - 1)  # Index of the window end

    # Create an array of float indices
    indices = np.arange(window_start, window_end + 1)
    indices = np.concatenate(([indices[0] - 0.5], indices, [indices[-1] + 0.5]))

    # Plot a shaded region for the window
    plt.fill_between(indices, min(Q[l][::2])/1.2, max(Q[l][::2])*1.2, color='gray', alpha=0.2)

    # plt.xlabel('Index')
    plt.ylabel('Flux')
    # plt.title('Flux Plot')
    plt.legend()

    # Plot for d[k][0]

    plt.subplot(2, 1, 2)
    plt.plot(range(len(d_g[k][l])), d_g[k][l],color='C0', linestyle='-',linewidth=1)
    matches = all_matches_g[k]
    for i, match_count in enumerate(matches):
        plt.text(i+0.2, d_g[k][l][i]+d_g[k][l][i]*5/100, str(match_count), fontsize=10, color='red', ha='left')


        # Plot dummy points with desired colors and markers
    plt.plot([], [], color='black', marker='o', label='Query chosed')
    plt.plot([], [], color='red', marker='x', label='missing cases !')
    plt.plot([], [], color='blue', marker='s', label='Matched here')
    plt.plot([], [], color='green', marker='*', label='Matches with a different `l` (Query)')
    plt.plot([], [], color='yellow', marker='^', label=f'Not matched with any of the {L_max} options we selected')

    for i, val in enumerate(R_l_g[k][l]):
            if val == -99:
                plt.scatter(i, d_g[k][l][i], color='red', marker='x', s=50)  # marker size 50
            elif val == l:
                plt.scatter(i, d_g[k][l][i], color='blue', marker='s', s=50)
            elif val == -2:
                plt.scatter(i, d_g[k][l][i], color='black', marker='o', s=50)  

            elif matches[i]==0:
                plt.scatter(i, d_g[k][l][i], color='yellow', marker='^', s=50)  
            else:
                plt.scatter(i, d_g[k][l][i], color='green', marker='*', s=75)




    plt.legend(fontsize=8) 

    plt.ylabel('distance')


    # # Plot for alphat[k][0]
    # plt.subplot(4, 1, 4)
    # plt.plot(range(len(alp[k][l][::2])), alp[k][l][::2], marker='.', linestyle='-',color='black')
    # plt.xlabel('Index')
    # plt.ylabel('alpha')
    # # plt.title('alpha Plot')


    plt.subplots_adjust(top=0.93)  # Adjust the top margin for the super title
    plt.suptitle(f"{objects[k]}", fontname='Arial', fontsize=16, fontweight='bold')

    plt.show()



In [ ]:
def plot_distance_flux_r(k,l):
    
    # # Plotting
    plt.figure(figsize=(15, 10))

    # Plot for F[k]
    plt.subplot(2, 1, 1)

    # plt.plot(range(int(len(F[k])/2)), F[k][::2], c='g', label='F[k] Even')
    #plt.plot(range(int(len(F[k])/2)), F[k][1::2], c='r', label='F[k] Odd')

    for i in range(int(len(F[objects[k]])/2)):
#         if source[objects[k]][2*i] == 0:
#             marker = 'x'
#             plt.errorbar(i, F[objects[k]][2*i], 
#                      #sig[objects[k]][2*i]*0.4,
#                      c='g', marker=marker)
#         else:
#             marker = 'o'
#             plt.errorbar(i, F[objects[k]][2*i], 
#                          sig[objects[k]][2*i]*0.4,
#                          c='g', marker=marker)

        if source[objects[k]][2*i+1] == 0:
            marker = 'x'
            plt.errorbar(i, F[objects[k]][2*i+1],
                     #sig[objects[k]][2*i+1]*0.4,
                     c='r', marker=marker)
        else:
            marker = 'o'
            plt.errorbar(i, F[objects[k]][2*i+1],
                     sig[objects[k]][2*i+1],
                     c='r', marker=marker)
            
    plt.plot([], [], color='red', marker='x', label='missing points !')
#     plt.plot([], [], color='green', marker='o', label='origin')
#     plt.plot([], [], color='green', marker='x', label='missing points !')
    plt.plot([], [], color='red', marker='o', label='origin')


#     plt.plot(range(int(len(F[objects[k]])/2)), F[objects[k]][::2], c='g', linewidth = 1)
    plt.plot(range(int(len(F[objects[k]])/2)), F[objects[k]][1::2], c='r', linewidth = 1)


#     plt.plot(range(int(len(Q[l])/2)), Q[l][::2], c='C0', label='Q[l]',marker='.')
    plt.plot(range(int(len(Q[l])/2)), Q[l][1::2], c='C1', label='Q[l]',marker='.')

    # Define the window of indices
    window_start = -0  # Index of the window start
    window_end = int(len(Q[l])/2 - 1)  # Index of the window end

    # Create an array of float indices
    indices = np.arange(window_start, window_end + 1)
    indices = np.concatenate(([indices[0] - 0.5], indices, [indices[-1] + 0.5]))

    # Plot a shaded region for the window
    plt.fill_between(indices, min(Q[l][1::2])/1.2, max(Q[l][1::2])*1.2, color='gray', alpha=0.2)

    # plt.xlabel('Index')
    plt.ylabel('Flux')
    # plt.title('Flux Plot')
    plt.legend()

    # Plot for d[k][0]

    plt.subplot(2, 1, 2)
    plt.plot(range(len(d_r[k][l])), d_r[k][l],color='C0', linestyle='-',linewidth=1)
    matches = all_matches_r[k]
#     for i, match_count in enumerate(matches):
#         print(match_count)
        #plt.text(i+0.2, d_r[k][l][i]+d_r[k][l][i]*5/100, str(match_count), fontsize=10, color='red', ha='left')


        # Plot dummy points with desired colors and markers
    plt.plot([], [], color='black', marker='o', label='Query chosed')
    plt.plot([], [], color='red', marker='x', label='missing cases !')
    plt.plot([], [], color='blue', marker='s', label='Matched here')
    plt.plot([], [], color='green', marker='*', label='Matches with a different `l` (Query)')
    plt.plot([], [], color='yellow', marker='^', label=f'Not matched with any of the {L_max} options we selected')

    for i, val in enumerate(R_l_r[k][l]):
            if val == -99:
                plt.scatter(i, d_r[k][l][i], color='red', marker='x', s=50)  # marker size 50
                plt.text(i+0.2, d_r[k][l][i]+d_r[k][l][i]*5/100, str(matches[i]), fontsize=10, color='red', ha='left')

            elif val == l:
                plt.scatter(i, d_r[k][l][i], color='blue', marker='s', s=50)
                plt.text(i+0.2, d_r[k][l][i]+d_r[k][l][i]*5/100, str(matches[i]), fontsize=10, color='blue', ha='left')

            elif val == -2:
                plt.scatter(i, d_r[k][l][i], color='black', marker='o', s=50) 
                plt.text(i+0.2, d_r[k][l][i]+d_r[k][l][i]*5/100, str(matches[i]), fontsize=10, color='black', ha='left')
            elif matches[i]==0:
                plt.scatter(i, d_r[k][l][i], color='yellow', marker='^', s=50) 
                plt.text(i+0.2, d_r[k][l][i]+d_r[k][l][i]*5/100, str(matches[i]), fontsize=10, color='yellow', ha='left')

            else:
                plt.scatter(i, d_r[k][l][i], color='green', marker='*', s=75)
                
                plt.text(i+0.2, d_r[k][l][i]+d_r[k][l][i]*5/100, str(matches[i]), fontsize=10, color='green', ha='left')





    plt.legend(fontsize=8) 

    plt.ylabel('distance')


    # # Plot for alphat[k][0]
    # plt.subplot(4, 1, 4)
    # plt.plot(range(len(alp[k][l][::2])), alp[k][l][::2], marker='.', linestyle='-',color='black')
    # plt.xlabel('Index')
    # plt.ylabel('alpha')
    # # plt.title('alpha Plot')


    plt.subplots_adjust(top=0.93)  # Adjust the top margin for the super title
    plt.suptitle(f"{objects[k]}, k= {k}, l = {l}", fontname='Arial', fontsize=16, fontweight='bold')

    plt.show()

